<a href="https://colab.research.google.com/github/DevBatra05/DeepLearningLab/blob/main/Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
def perceptron_and(x1, x2):
    w1, w2, bias = 1, 1, -1
    z = w1*x1 + w2*x2 + bias
    return 1 if z > 0 else 0

# Verify all input combinations
for x1, x2 in [(0,0),(0,1),(1,0),(1,1)]:
    print(f"{x1},{x2} -> {perceptron_and(x1,x2)}")


0,0 -> 0
0,1 -> 0
1,0 -> 0
1,1 -> 1


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# XOR dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=float)
Y = np.array([[0],[1],[1],[0]], dtype=float)

# Define a small MLP: 2 inputs -> 2 hidden -> 1 output
model = Sequential([
    Dense(2, activation='sigmoid', input_shape=(2,)),  # hidden layer
    Dense(1, activation='sigmoid')                    # output layer
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, Y, epochs=500, verbose=0)
preds = (model.predict(X) > 0.5).astype(int)
print("Predictions:", preds.T, "Actual:", Y.T)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predictions: [[0 1 1 0]] Actual: [[0. 1. 1. 0.]]


In [7]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import kagglehub

# Download dataset
data_path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Dataset Path:", data_path)

# Go inside "leapGestRecog" folder if present
if "leapGestRecog" in os.listdir(data_path):
    data_path = os.path.join(data_path, "leapGestRecog")

print("Using data folder:", data_path)

# Load dataset
X, y = [], []
for person in os.listdir(data_path):
    person_path = os.path.join(data_path, person)
    if not os.path.isdir(person_path):
        continue
    for gesture in os.listdir(person_path):
        gesture_path = os.path.join(person_path, gesture)
        if not os.path.isdir(gesture_path):
            continue
        for img_file in os.listdir(gesture_path):
            img_path = os.path.join(gesture_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale
            if img is None:
                continue
            img = cv2.resize(img, (50, 50))                  # resize to 50x50
            X.append(img.flatten())                          # flatten into 1D
            y.append(gesture)                                # label = gesture name

X = np.array(X, dtype="float32") / 255.0  # normalize
y = np.array(y)

print("Dataset loaded:", X.shape, y.shape)

# One-hot encode labels
encoder = LabelBinarizer()
y = encoder.fit_transform(y)
print("Classes:", encoder.classes_)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Build MLP model
model = Sequential([
    Dense(512, activation='relu', input_shape=(2500,)),  # 50*50=2500 features
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(y.shape[1], activation='softmax')              # number of gesture classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")

# Predictions
preds = np.argmax(model.predict(X_test[:10]), axis=1)
actual = np.argmax(y_test[:10], axis=1)
print("Predictions:", preds)
print("Actual:", actual)


Using Colab cache for faster access to the 'leapgestrecog' dataset.
Dataset Path: /kaggle/input/leapgestrecog
Using data folder: /kaggle/input/leapgestrecog/leapGestRecog
Dataset loaded: (20000, 2500) (20000,)
Classes: ['01_palm' '02_l' '03_fist' '04_fist_moved' '05_thumb' '06_index' '07_ok'
 '08_palm_moved' '09_c' '10_down']


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.5599 - loss: 1.2956 - val_accuracy: 0.9822 - val_loss: 0.1021
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9646 - loss: 0.1379 - val_accuracy: 0.9966 - val_loss: 0.0263
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9837 - loss: 0.0602 - val_accuracy: 0.9984 - val_loss: 0.0119
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9860 - loss: 0.0434 - val_accuracy: 0.9984 - val_loss: 0.0084
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9898 - loss: 0.0345 - val_accuracy: 0.9987 - val_loss: 0.0087
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9911 - loss: 0.0296 - val_accuracy: 0.9997 - val_loss: 0.0037
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9935 - loss: 0.0198 - val_accuracy: 0.9991 - val_loss: 0.0031
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9940 - loss: 0.0166 - val_accu